In [6]:
from sympy import Rational, factorial, zeros
from sympy.matrices import Matrix
import sympy as sp

def padee_approx(M, m):
    """
    Compute the mth degree Pade approximation for e^M.
    
    Parameters:
    M : matrix or scalar
        The input matrix or scalar for which to compute the exponential.
    m : int
        The degree of the Pade approximation.
        
    Returns:
    Matrix or scalar
        The Pade approximation of e^M.
    """
    # Numerator Coefficients
    def B_coef(k):
        return Rational(
            factorial(2*m - k) * factorial(m),
            factorial(2*m) * factorial(k) * factorial(m - k)
        )
    
    # Denominator Coefficients    
    def A_coef(k): 
        return (-1)**k * B_coef(k)  
    
    # Check if M is a matrix by verifying if it has the 'shape' attribute
    if hasattr(M, 'shape'):
        # M is assumed to be a matrix
        rows, cols = M.shape
        N = zeros(rows, cols)
        D = zeros(rows, cols)
    
        for k in range(0, m + 1):
            N += B_coef(k) * M**k
            D += A_coef(k) * M**k
        
        # Return the matrix product of N and the inverse of D
        return N * D.inv()
    else:
        # M is assumed to be a scalar
        N = 0
        D = 0
        for k in range(0, m + 1):
            N += B_coef(k) * M**k
            D += A_coef(k) * M**k
        
        # Return the scalar division of N by D
        return N / D


In [8]:
display( sp.simplify(padee_approx(sp.symbols('B'),3))) # Only do odd numbers, even numbers introduce poles or something that make them less accurate.

(-B**3 - 12*B**2 - 60*B - 120)/(B**3 - 12*B**2 + 60*B - 120)

In [4]:
theta_for_m_3 = 1.495585217958292 * 10**-2 
theta_for_m_5 = 2.539398330063230 * 10**-1

In [5]:
import sympy as sp

def compute_s(M, theta):
    """
    Compute the scaling factor 's' based on the infinity norm of matrix M and threshold theta.

    Parameters:
        M (sp.Matrix): The input matrix.
        theta (float): The threshold value.

    Returns:
        int: The scaling factor 's'.
    """
    norm = M.norm(ord=sp.oo)
    s = sp.ceiling(sp.log(norm / theta, 2))
    s = max(s, 0)  # Ensure s is not negative. YOU may be able to get rid of this by ensuring the norm is never less than the theta. 
    return s

def scale_matrix(M, theta):
    """
    Scale the matrix M by 2^s, where s is determined based on theta.

    Parameters:
        M (sp.Matrix): The input matrix.
        theta (float): The threshold value.

    Returns:
        tuple: A tuple containing the scaled matrix B and the scaling factor s.
    """
    s = compute_s(M, theta)
    B = M / (2 ** s)
    return B, s

def padee_and_scale(M, theta_for_m_3):
    """
    Apply Pade approximation to matrix M, scaling it if necessary based on theta_for_m_3.

    Parameters:
        M (array-like): The input matrix.
        theta_for_m_3 (float): The threshold value for deciding scaling.

    Returns:
        sp.Matrix: The resulting matrix after Pade approximation and scaling.
    """
    # Ensure M is a SymPy Matrix with complex entries
    M = sp.Matrix(M).applyfunc(lambda x: sp.nsimplify(x, rational=True))  # Convert to SymPy types

    # Compute the infinity norm of M
    norm_inf = M.norm(ord=sp.oo)

    if norm_inf < theta_for_m_3:
        return padee_approx(M, 3)

    # Scale the matrix if norm_inf >= theta_for_m_3
    B, s = scale_matrix(M, theta_for_m_3)

    # Apply Pade approximation to the scaled matrix
    B = padee_approx(B, 3)

    # Rescale the result
    B = B ** (2 ** s)

    return B


In [6]:
from numba import jit
import numpy as np


testMatrix = Matrix([[1+3*sp.I,0.2+0.4*sp.I],[-4-4*sp.I,-0.8*sp.I]])

display("The inputted matrix:", testMatrix)

print("The matrices norm is", testMatrix.norm(sp.oo).evalf(3))

display("In-built function exp():", testMatrix.exp().evalf(3))

display("My function:", padee_and_scale(testMatrix, theta_for_m_3).evalf(3))








'The inputted matrix:'

Matrix([
[ 1 + 3*I, 0.2 + 0.4*I],
[-4 - 4*I,      -0.8*I]])

The matrices norm is 6.46


'In-built function exp():'

Matrix([
[-1.17 + 1.33*I, -0.235 + 0.397*I],
[ 1.23 - 5.71*I,    1.57 - 1.67*I]])

'My function:'

Matrix([
[-1.17 + 1.33*I, -0.235 + 0.397*I],
[ 1.23 - 5.71*I,    1.57 - 1.67*I]])

# So the pipeline for this is gonna look like this:

1. We spilt the Hamilation Matrix into 6 matrices, one associasted with each cell so we have empty matrices except for one cell. This minimizes their norm  and allows for much easier matrix multiplication and squaring so computation is easier. (Here H1 H2 etc mean the six matrices that add up to H)
    $$e^H = e^{H_1}e^{H_2}e^{H_3}e^{H_4}e^{H_5}e^{H_6}$$
2. In VHDL we take the symbolic equations for each Matrices cell and use them to calculate a concrete value (example from one of the matrices):
    $$-it(-H1+H2-J) = C$$
3. We take this matrix with one cell that is the value C and if the matrices norm exceeds $1.496* 10^{-2}$ we calculate our scaling factor s, if not we simply use it directly with Padee approximation. 
    $$
    s = \left\lceil \log_2\left(\frac{\|A\|}{\theta}\right) \right\rceil , \theta=1.496* 10^{-2}
    $$
4. If we have a scaling factor $s$ we divide the matrix with value $C$ by the value $2^s$ to get a new scaled down matrix.
$$B = \frac{H_n}{2^{s}} = \frac{C}{2^{s}} $$
5. We use this scaled down matrix B in our third degree Pradee approximattion. 
$$\frac{- B^{3} - 12 B^{2} - 60 B - 120}{B^{3} - 12 B^{2} + 60 C - 120} = B_n$$
6. We take this matrix $B_n$ and scale it back up through the following formula:
$$B_f = B_n^{2^{s}}$$
7. $Bf$ is our final result. We do this $H_1$ through $H_6$ and then multiply all six together to get our final result $e^H$

